In [88]:
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from lifelines.statistics import logrank_test
from lifelines import CoxPHFitter
import warnings
warnings.filterwarnings("ignore")
from statsmodels.stats.multitest import multipletests
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import NMF

In [52]:
full_data = pd.read_csv('Full Data.txt', index_col = [0])

In [53]:
full_data

,gender,ajcc_pathologic_tumor_stage,tumor_status,DSS,DSS.time,TP53,TTN,FAT1,MUC16,CSMD3,...,NIPBL,CEACAM5,CEACAM6,CLCA4,EGFR,Missense_Mutation,Nonsense_Mutation,Nonstop_Mutation,Splice_Site,age
patient_id,,,,,,,,,,,,,,,,,,,,,
TCGA-4P-AA8J,1,0,0,0.0,0.279452,822.678,1157.3300,1620.71,1004.4700,1.2325,...,1595.440,56.0776,423.3550,502.2340,722.847,92,8,0,2,1
TCGA-BA-4074,1,0,1,1.0,1.265753,1805.510,103.6870,6415.45,17.3422,0.3854,...,1605.890,153.3820,275.9330,12.7176,1267.900,106,8,0,1,1
TCGA-BA-4075,1,1,1,1.0,0.775342,383.760,295.9180,4162.40,1.0720,36.9824,...,907.410,18.2232,45.5581,10.1836,4833.430,113,10,0,4,0
TCGA-BA-4077,0,0,1,1.0,3.106849,1929.260,207.3870,7246.87,432.6270,0.0000,...,1174.690,2392.6900,2437.9200,720.8850,2371.940,234,20,0,6,0
TCGA-BA-5149,1,0,1,1.0,2.208219,165.298,347.9540,8848.73,110.5410,0.3422,...,1798.770,37.3032,447.6390,5.4757,6934.280,96,6,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-UF-A7JO,0,0,1,1.0,1.728767,1604.880,176.1930,1881.36,347.7040,2.8008,...,854.256,1498.8500,2014.6000,82.8248,530.911,135,9,0,4,1
TCGA-UF-A7JS,1,0,1,1.0,1.863014,2457.400,1379.4300,1339.31,243.6470,19.4320,...,982.063,358.7440,294.4690,32.8849,246.637,50,1,0,1,1
TCGA-UF-A7JT,0,0,1,1.0,2.720548,1684.850,149.5190,3430.74,213.8530,14.2857,...,1009.520,49.7835,209.5240,200.4330,790.905,152,10,0,3,1


In [ ]:
time_limit = range(1,11)
n_components = 3  # Specify the number of components
nmf = NMF(n_components=n_components)

In [96]:
def classifcation_svm(df):
    # get features you select
    X = df.drop(columns = ['DSS.time', 'DSS'])
    y = df['DSS']

    # Split the dataset into training and testing datasets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    # SVM
   
    # Define the model
    svm_clf = svm.SVC()

    # Train the model
    svm_clf.fit(X_train, y_train)
   
    # Predict on the test set
    svm_predictions = svm_clf.predict(X_test)

    # Evaluate the model
    return accuracy_score(y_test, svm_predictions)
    #print("SVM Accuracy:", accuracy_score(y_test, svm_predictions))

In [101]:
for i in time_limit:
    data = full_data[full_data['DSS.time'] <= i]
    X_reduced = nmf.fit_transform(data)
    df_reduced = pd.DataFrame(X_reduced, columns= ['component_1', 'component_2', 'component_3'])
    df_reduced['DSS'] = full_data['DSS'].reset_index()['DSS']
    df_reduced['DSS.time'] = full_data['DSS.time'].reset_index()['DSS.time']
    lis = []
    for k in range(100):
        accuracy = classifcation_svm(df_reduced)
        lis.append(accuracy)
    mean_acc = sum(lis)/len(lis)
    print('For time limit ' + str(i) + ' and nmf features')
    print('SVM Accuracy ' + str(mean_acc))

For time limit 1 and nmf features
SVM Accuracy 0.7327272727272728
For time limit 2 and nmf features
SVM Accuracy 0.6998076923076922
For time limit 3 and nmf features
SVM Accuracy 0.669393939393939
For time limit 4 and nmf features
SVM Accuracy 0.7045205479452059
For time limit 5 and nmf features
SVM Accuracy 0.721125
For time limit 6 and nmf features
SVM Accuracy 0.7057831325301204
For time limit 7 and nmf features
SVM Accuracy 0.708235294117647
For time limit 8 and nmf features
SVM Accuracy 0.7124137931034484
For time limit 9 and nmf features
SVM Accuracy 0.7118181818181819
For time limit 10 and nmf features
SVM Accuracy 0.706590909090909


In [102]:
def classifcation_randomforest(df):
    # get features you select
    X = df.drop(columns = ['DSS.time', 'DSS'])
    y = df['DSS']

    # Split the dataset into training and testing datasets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    # add classification
    # choose any classfication to do prediction
    # Define the model
    rf_clf = RandomForestClassifier()

    # Train the model
    rf_clf.fit(X_train, y_train)
    # Predict on the test set
    rf_predictions = rf_clf.predict(X_test)

    # Evaluate the model
    return accuracy_score(y_test, rf_predictions)
    #print("Random Forest Accuracy:", accuracy_score(y_test, rf_predictions))

In [104]:
for i in time_limit:
    data = full_data[full_data['DSS.time'] <= i]
    X_reduced = nmf.fit_transform(data)
    df_reduced = pd.DataFrame(X_reduced, columns= ['component_1', 'component_2', 'component_3'])
    df_reduced['DSS'] = full_data['DSS'].reset_index()['DSS']
    df_reduced['DSS.time'] = full_data['DSS.time'].reset_index()['DSS.time']
    lis = []
    for k in range(100):
        accuracy = classifcation_randomforest(df_reduced)
        lis.append(accuracy)
    mean_acc = sum(lis)/len(lis)
    print('For time limit ' + str(i) + ' and nmf features')
    print('Random Forest Accuracy ' + str(mean_acc))

For time limit 1 and nmf features
Random Forest Accuracy 0.6813636363636363
For time limit 2 and nmf features
Random Forest Accuracy 0.6453846153846153
For time limit 3 and nmf features
Random Forest Accuracy 0.6048484848484853
For time limit 4 and nmf features
Random Forest Accuracy 0.6698630136986304
For time limit 5 and nmf features
Random Forest Accuracy 0.6445000000000001
For time limit 6 and nmf features
Random Forest Accuracy 0.6609638554216869
For time limit 7 and nmf features
Random Forest Accuracy 0.6238823529411767
For time limit 8 and nmf features
Random Forest Accuracy 0.6509195402298852
For time limit 9 and nmf features
Random Forest Accuracy 0.6334090909090907
For time limit 10 and nmf features
Random Forest Accuracy 0.660113636363636


In [105]:
def classifcation_logistic(df):
    # get features you select
    X = df.drop(columns = ['DSS.time', 'DSS'])
    y = df['DSS']

    # Split the dataset into training and testing datasets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    # add classification
    # choose any classfication to do prediction
    # Define the model
    log_reg = LogisticRegression()

    # Train the model
    log_reg.fit(X_train, y_train)
    log_reg_predictions = log_reg.predict(X_test)

    # Evaluate the model
    return accuracy_score(y_test, log_reg_predictions)
    #print("Logistic Regression Accuracy:", accuracy_score(y_test, log_reg_predictions))

In [106]:
for i in time_limit:
    data = full_data[full_data['DSS.time'] <= i]
    X_reduced = nmf.fit_transform(data)
    df_reduced = pd.DataFrame(X_reduced, columns= ['component_1', 'component_2', 'component_3'])
    df_reduced['DSS'] = full_data['DSS'].reset_index()['DSS']
    df_reduced['DSS.time'] = full_data['DSS.time'].reset_index()['DSS.time']
    lis = []
    for k in range(100):
        accuracy = classifcation_logistic(df_reduced)
        lis.append(accuracy)
    mean_acc = sum(lis)/len(lis)
    print('For time limit ' + str(i) + ' and nmf features')
    print('Logistic Accuracy ' + str(mean_acc))

For time limit 1 and nmf features
Logistic Accuracy 0.7177272727272728
For time limit 2 and nmf features
Logistic Accuracy 0.6998076923076922
For time limit 3 and nmf features
Logistic Accuracy 0.6703030303030303
For time limit 4 and nmf features
Logistic Accuracy 0.7028767123287674
For time limit 5 and nmf features
Logistic Accuracy 0.7039999999999996
For time limit 6 and nmf features
Logistic Accuracy 0.7084337349397586
For time limit 7 and nmf features
Logistic Accuracy 0.707882352941176
For time limit 8 and nmf features
Logistic Accuracy 0.7018390804597704
For time limit 9 and nmf features
Logistic Accuracy 0.7153409090909092
For time limit 10 and nmf features
Logistic Accuracy 0.7093181818181814


# Log-rank test

In [22]:
def quantile_gene(df, x):
    q1 = df[x].quantile(0.1)
    q3 = df[x].quantile(0.9)
    df[x + 'binary'] = df[x].apply(lambda x: 0 if x <= q1 else 1 if x >= q3 else np.nan)

In [23]:
def logrank(df, lis,time):
    dic ={}
    data = df[df['DSS.time'] <= time]
    for i in lis:
        quantile_gene(data, i)
        low_expression = data[data[i + 'binary'] == 0]
        high_expression =  data[data[i +'binary'] == 1]
        # logrank
        results = logrank_test(high_expression['DSS.time'], low_expression['DSS.time'],
                       high_expression['DSS'], low_expression['DSS'])
        z_value = results.test_statistic
        if z_value > 1.96 or z_value < -1.96:
            dic[i] = z_value
    # return a dictionary of name and z value
    return dic

In [9]:
gene_expression = list(full_data.drop(columns = ['gender', 'ajcc_pathologic_tumor_stage', 'tumor_status', 'DSS' ,
                                      'DSS.time', 'Missense_Mutation', 'Nonsense_Mutation', 
                                      'Nonstop_Mutation', 'Splice_Site', 'age']).columns)

In [10]:
# for gene expression
logrank(full_data,gene_expression, 2) # example: use full data and 2 years as limit

{'TP53': 2.120617453561617,
 'TTN': 2.143271855534282,
 'CDKN2A': 3.1481755529943243,
 'PIK3CA': 4.680473121342188,
 'CASP8': 3.389085031356897,
 'XIRP2': 3.1703821360088784,
 'CUBN': 4.046813671678665,
 'MUC17': 3.646690183663279,
 'MUC5B': 2.0729664690136795,
 'PCDH15': 2.6959108383640076,
 'PCDH11X': 2.5834320206641146,
 'AHNAK2': 2.214970513627507,
 'FMN2': 2.615400302580672,
 'PKHD1': 2.104351556763302,
 'CDH10': 3.070610713909658,
 'EP300': 3.1299077752441216,
 'ANK2': 2.7759016074401215,
 'RYR1': 2.9064141544880564,
 'VCAN': 2.7374238783551403,
 'EGFR': 3.7419164805770855}

In [302]:
#select top n genes by sort z values as features from full_data

In [11]:
mutation = ['Missense_Mutation', 'Nonsense_Mutation', 'Nonstop_Mutation', 'Splice_Site']

In [12]:
# for mutation
logrank(full_data,mutation, 2) # example: use full data and 2 years as limit

{'Missense_Mutation': 2.2088500784074974,
 'Nonsense_Mutation': 1.9839793008189441,
 'Splice_Site': 5.71020812335661}

# Cox model

In [13]:
def coxfitter(data, duration, event):
    # Define the CoxPHFitter object
    cph = CoxPHFitter()
    # Fit the model to your data
    cph.fit(data, duration_col= duration, event_col=event)
    summary = cph.summary
    return summary

In [14]:
def cox_model(df, time, lis):
    data = df[df['DSS.time'] <= time]
    signi = []
    pvals = []
    pvals_correct = []
    for i in lis:
        summary = coxfitter(data[[i ,'gender','ajcc_pathologic_tumor_stage', 
                                   'tumor_status', 'age', 'DSS.time', 'DSS']], 'DSS.time', 'DSS')
        p_values = summary["p"]
        rejected, pvals_corrected = multipletests(p_values, method='fdr_bh')[:2]
        summary['p_corrected'] = pvals_corrected
        p = summary.loc[i, "p"]
        p_correct = summary.loc[i, "p_corrected"]
        pvals.append(p)
        pvals_correct.append(p_correct)
        signi.append(rejected[0])
    features = [col for col in lis]  # list of features
    # Create a DataFrame with the results
    results = pd.DataFrame({'Features': features,'p': pvals,'p_corrected': pvals_correct, 'significant': signi})
    return results[results['significant'] == True]


In [15]:
# gene expression level
cox_model(full_data, 2, gene_expression )# example: use full data and 2 years as limit

,Features,p,p_corrected,significant
19,CASP8,2.297870e-03,1.148935e-02,True
42,DNAH9,3.421607e-03,1.710804e-02,True
43,DNAH11,3.413931e-03,1.706965e-02,True
46,LRP2,1.491141e-38,7.455703e-38,True
54,RP1,2.284010e-03,1.142005e-02,True
59,DNAH7,7.134398e-03,3.567199e-02,True
83,MDN1,8.718074e-03,4.359037e-02,True


In [16]:
# mutation
cox_model(full_data, 2, mutation)# example: use full data and 2 years as limit

,Features,p,p_corrected,significant
0,Missense_Mutation,4.747304e-08,2.373652e-07,True
1,Nonsense_Mutation,3.141682e-04,1.570841e-03,True
2,Nonstop_Mutation,8.020092e-03,4.010046e-02,True
3,Splice_Site,3.520589e-05,1.760294e-04,True


# Classification

In [318]:
#select top n mutations by z values as features columns from full_data

In [60]:
#def classifcation_svm(df, features):
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from lifelines.statistics import logrank_test
from lifelines import CoxPHFitter
import warnings
warnings.filterwarnings("ignore")
from statsmodels.stats.multitest import multipletests
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
# get features you select
X = X_reduced
y = full_data['DSS']

# Split the dataset into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# SVM

# Define the model
svm_clf = svm.SVC()

# Train the model
svm_clf.fit(X_train, y_train)

# Predict on the test set
svm_predictions = svm_clf.predict(X_test)

# Evaluate the model
#return accuracy_score(y_test, svm_predictions)
print("SVM Accuracy:", accuracy_score(y_test, svm_predictions))

SVM Accuracy: 0.6888888888888889


In [30]:
X = full_data
y = full_data['DSS']

# Split the dataset into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# SVM

# Define the model
svm_clf = svm.SVC()

# Train the model
svm_clf.fit(X_train, y_train)

# Predict on the test set
svm_predictions = svm_clf.predict(X_test)

# Evaluate the model
#return accuracy_score(y_test, svm_predictions)
print("SVM Accuracy:", accuracy_score(y_test, svm_predictions))

SVM Accuracy: 0.7333333333333333


In [ ]:
for i in time_limit:
    lis = []
    for k in range(10000):
        accuracy = classifcation_svm(full_data, feature_dic[i])
        lis.append(accuracy)
    mean_acc = sum(lis)/len(lis)
    print('For time limit ' + str(i) + ' and gene expression features from Cox model')
    print('SVM Accuracy ' + str(mean_acc))

In [6]:
pip install deepsurv

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 2.8 MB 16.0 MB/s eta 0:00:01
  Created wheel for deepsurv: filename=deepsurv-0.1.0-py3-none-any.whl size=11163 sha256=466c38c2e46dc5e789c9976ecd16fed8d462583876c293cbdd523917f6f981fa
  Stored in directory: /tmp/pip-ephem-wheel-cache-n70xzz7s/wheels/8b/35/64/cb4fa77f7de25e6133c42cc1749ecd28b82131fcc937ab37ee
  Created wheel for theano: filename=Theano-1.0.5-py3-none-any.whl size=2668094 sha256=ada06ea1dd9034eedd839a505e6ab6f1cb2b0b5f3fd8f608599e6f4c3ea374b4
  Stored in directory: /tmp/pip-ephem-wheel-cache-n70xzz7s/wheels/84/cb/19/235b5b10d89b4621f685112f8762681570a9fa14dc1ce904d9
Successfully built deepsurv theano
Note: you may need to restart the kernel to use updated packages.


In [27]:
from lifelines import CoxPHFitter
from lifelines.utils import k_fold_cross_validation
from lifelines.utils import concordance_index

import numpy as np
import pandas as pd
X = full_data[['gender', 'ajcc_pathologic_tumor_stage','DSS', 'tumor_status','TP53',	'TTN',	'FAT1',	'MUC16','CLCA4','EGFR','Missense_Mutation',	'Nonsense_Mutation',	'Nonstop_Mutation',	'Splice_Site',	'age']]  # Replace with the actual feature columns
y = full_data['DSS.time']  # Replace with the column representing time-to-event
# Define the Cox-Nnet model
cox_nnet_model = CoxPHFitter(n_iter=1000, l2_reg=0.01)

# Fit the model to the data
cox_nnet_model.fit(X, 'survival_time', event_col='DSS', show_progress=True)  # Replace 'event' with your actual event indicator column

# Calculate the concordance index (C-index) as a measure of model performance
c_index = concordance_index(cox_nnet_model.durations, -cox_nnet_model.predict_partial_hazard(cox_nnet_model.data).values.flatten(), cox_nnet_model.event_observed)
print(f'C-index: {c_index}')

# Perform k-fold cross-validation
scores = k_fold_cross_validation(cox_nnet_model, X, 'survival_time', event_col='DSS', k=5)  # Replace 'event' with your actual event indicator column
average_score = np.mean(scores)
print(f'Average cross-validated C-index: {average_score}')
'''
E = full_data['DSS']  # Replace with the column representing event indicator


# Split the dataset into training and testing sets
X_train, X_test, T_train, T_test, E_train, E_test = train_test_split(
    X, T, E, test_size=0.2, seed=0
)

# Define the DeepSurv model architecture
hyperparams = {'L2_reg': 1e-4, 'batch_norm': True, 'dropout': 0.2}
architecture = [
    {'activation': 'relu', 'num_units': 100},
    {'activation': 'relu', 'num_units': 100},
]

# Train the DeepSurv model
model = deep_surv.DeepSurv(architecture=architecture, hyperparams=hyperparams)
model.fit(X_train, T_train, E_train, num_epochs=100)

# Evaluate the model
c_index = model.concordance_index(X_test, T_test, E_test)
#print("Concordance Index:", c_index)
'''


TypeError: __init__() got an unexpected keyword argument 'n_iter'

In [1]:
pip install h5py>=2.7.0

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install logger==1.4

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Preparing metadata (setup.py) ... done
  Created wheel for logger: filename=logger-1.4-py3-none-any.whl size=1758 sha256=d8ce13cc2ecfbb5bbe60c36468df1670cecaf6f1a02d041bf3350da082d17952
  Stored in directory: /tmp/pip-ephem-wheel-cache-wmjqlnmy/wheels/4c/62/b0/eb01dd1d29b9daccb651817618f8846e40384e45833f6aaf6d
Successfully built logger
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install Optunity>=1.1.1

Note: you may need to restart the kernel to use updated packages.


In [23]:
import theano 


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/gputest/.conda/envs/torch/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_29988/3470119688.py", line 1, in <module>
    import theano
  File "/home/gputest/.conda/envs/torch/lib/python3.8/site-packages/theano/__init__.py", line 76, in <module>
    from theano.scan_module import scan, map, reduce, foldl, foldr, clone
  File "/home/gputest/.conda/envs/torch/lib/python3.8/site-packages/theano/scan_module/__init__.py", line 40, in <module>
    from theano.scan_module import scan_opt
  File "/home/gputest/.conda/envs/torch/lib/python3.8/site-packages/theano/scan_module/scan_opt.py", line 59, in <module>
    from theano import tensor, scalar
  File "/home/gputest/.conda/envs/torch/lib/python3.8/site-packages/theano/tensor/__init__.py", line 6, in <module>
    from theano.tensor.basic import *
  File "/home/gputest/.conda/envs/torc

In [20]:

conda install m2w64-toolchain


Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - m2w64-toolchain

Current channels:

  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.



Note: you may need to restart the kernel to use updated packages.


In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from lifelines import CoxPHFitter
from keras.models import Sequential
from keras.layers import Dense

# Load and preprocess the dataset
dataset = pd.read_csv('Full Data.txt')
#X = dataset.drop(['DSS.time', 'DSS'], axis=1)
X = dataset[['gender', 'ajcc_pathologic_tumor_stage', 'tumor_status','TP53','TTN','FAT1','MUC16','CLCA4','EGFR','Missense_Mutation',	'Nonsense_Mutation',	'Nonstop_Mutation',	'Splice_Site',	'age']]  # Replace with the actual feature columns

y = dataset[['DSS.time', 'DSS']]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define and train the feedforward neural network
input_dim = X_train_scaled.shape[1]
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=input_dim))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train_scaled, y_train['DSS.time'], epochs=10, batch_size=32)

# Extract the learned representations from the hidden layers
hidden_layer_outputs = model.predict(X_train_scaled)

# Combine the learned representations with the survival data
X_train_combined = np.concatenate((X_train_scaled, hidden_layer_outputs), axis=1)

# Fit the Cox proportional hazards model
cox_nnet = CoxPHFitter()
cox_nnet.fit(pd.DataFrame(X_train_combined), duration_col=dataset[['DSS.time']], event_col=dataset[['DSS']])

# Predict survival probabilities for the test set
hidden_layer_outputs_test = model.predict(X_test_scaled)
X_test_combined = np.concatenate((X_test_scaled, hidden_layer_outputs_test), axis=1)
survival_probs = cox_nnet.predict_survival_function(pd.DataFrame(X_test_combined))

# Evaluate the model
c_index = cox_nnet.score(pd.DataFrame(X_test_combined), y_test, scoring_method="concordance_index")
print("Concordance Index: ", c_index)


Epoch 1/10
12/12 [==============================] - 1s 1ms/step - loss: 10.5587
Epoch 2/10
12/12 [==============================] - 0s 2ms/step - loss: 8.3932
Epoch 3/10
12/12 [==============================] - 0s 2ms/step - loss: 7.3197
Epoch 4/10
12/12 [==============================] - 0s 2ms/step - loss: 6.6885
Epoch 5/10
12/12 [==============================] - 0s 1ms/step - loss: 6.2632
Epoch 6/10
12/12 [==============================] - 0s 2ms/step - loss: 5.9638
Epoch 7/10
12/12 [==============================] - 0s 2ms/step - loss: 5.7065
Epoch 8/10
12/12 [==============================] - 0s 2ms/step - loss: 5.5584
Epoch 9/10
12/12 [==============================] - 0s 2ms/step - loss: 5.4240
Epoch 10/10
12/12 [==============================] - 0s 1ms/step


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [7]:
import numpy as np
from deepsurv import deep_surv
from deepsurv.datasets import load_rgbsg
from deepsurv.utils import train_test_split

SyntaxError: Missing parentheses in call to 'print'. Did you mean print('[INFO] Training CoxMLP')? (deep_surv.py, line 326)

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Assuming you have already prepared your dataset
# X: Features (input variables)
# y: Target variable (clinical outcome)
X = dataset[['gender', 'ajcc_pathologic_tumor_stage', 'tumor_status','TP53','TTN','FAT1','MUC16','CLCA4','EGFR','Missense_Mutation',	'Nonsense_Mutation',	'Nonstop_Mutation',	'Splice_Site',	'age']]  # Replace with the actual feature columns

y = dataset[['DSS.time', 'DSS']]
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the XGBoost classifier
model = xgb.XGBClassifier()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))


In [13]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Assuming you have already prepared your dataset
# X: Features (input variables)
# y_time: Survival time
# y_event: Event indicator (0: censored, 1: event occurred)
X = dataset[['gender', 'ajcc_pathologic_tumor_stage', 'tumor_status','TP53','TTN','FAT1','MUC16','CLCA4','EGFR','Missense_Mutation',	'Nonsense_Mutation',	'Nonstop_Mutation',	'Splice_Site',	'age']]  # Replace with the actual feature columns

y_time = dataset[['DSS.time']]
y_event= dataset[['DSS']]
# Split the data into training and testing sets
X_train, X_test, y_time_train, y_time_test, y_event_train, y_event_test = train_test_split(
    X, y_time, y_event, test_size=0.2, random_state=42)

# Initialize the XGBoost regressor
model = xgb.XGBRegressor(objective='survival:cox', eval_metric='cox-nloglik')

# Train the model
model.fit(X_train, y_time_train, event=y_event_train)

# Make predictions on the test set
predictions = model.predict(X_test)

# Calculate the concordance index (C-index)
cindex = model.score(X_test, y_time_test, y_event_test)
print("Concordance Index (C-index):", cindex)

# Calculate the mean squared error (MSE)
mse = mean_squared_error(y_time_test, predictions)
print("Mean Squared Error (MSE):", mse)


TypeError: fit() got an unexpected keyword argument 'event'

In [16]:
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from lifelines.utils import concordance_index
# Assuming you have already prepared your dataset
# X: Features (input variables)
# y_time: Survival time
# y_event: Event indicator (0: censored, 1: event occurred)
X = dataset[['gender', 'ajcc_pathologic_tumor_stage', 'tumor_status','TP53','TTN','FAT1','MUC16','CLCA4','EGFR','Missense_Mutation',	'Nonsense_Mutation',	'Nonstop_Mutation',	'Splice_Site',	'age']]  # Replace with the actual feature columns

# Split the data into training and testing sets
X_train, X_test, y_time_train, y_time_test, y_event_train, y_event_test = train_test_split(
    X, y_time, y_event, test_size=0.2, random_state=42)

# Convert the survival time and event indicator to the required format
y_train = np.column_stack((y_time_train, y_event_train))
y_test = np.column_stack((y_time_test, y_event_test))

# Define the custom loss function for survival analysis
def cox_loss(y_pred, y):
    # Extract the survival time and event indicator
    y_time = y[:, 0]
    y_event = y[:, 1]
    # Compute the negative log partial likelihood
    hazard_ratio = np.exp(y_pred)
    log_risk = np.log(np.cumsum(hazard_ratio))
    log_lik = y_pred - log_risk
    neg_log_lik = -np.sum(log_lik * y_event - log_risk * np.sum(y_event))
    return neg_log_lik

# Initialize the XGBoost regressor
model = xgb.XGBRegressor(objective=cox_loss)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate the concordance index (C-index)
cindex = concordance_index(y_time_test, -y_pred, y_event_test)
print("Concordance Index (C-index):", cindex)

ValueError: operands could not be broadcast together with shapes (720,) (360,) 

In [10]:
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from lifelines.utils import concordance_index
y = full_data[['DSS']]
def cox_loss(y_pred, y):
    # Extract the survival time and event indicator
    # Compute the negative log partial likelihood
    hazard_ratio = np.exp(y_pred)
    log_risk = np.log(np.cumsum(hazard_ratio))
    log_lik = y_pred - log_risk
    neg_log_lik = -np.sum(log_lik * y - log_risk * np.sum(y))
    return neg_log_lik

X = full_data[['gender', 'ajcc_pathologic_tumor_stage', 'tumor_status','TP53','TTN','FAT1','MUC16','CLCA4','EGFR','Missense_Mutation',	'Nonsense_Mutation',	'Nonstop_Mutation',	'Splice_Site',	'age']]  # Replace with the actual feature columns

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Initialize the XGBoost regressor
model = xgb.XGBRegressor(objective=cox_loss)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate the concordance index (C-index)
cindex = concordance_index(ytest, -y_pred, y_test)
print("Concordance Index (C-index):", cindex)

TypeError: cannot unpack non-iterable numpy.float32 object

In [12]:
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from lifelines.utils import concordance_index

y = full_data[['DSS']].values  # Convert to NumPy array
X = full_data[['gender', 'ajcc_pathologic_tumor_stage', 'tumor_status','TP53','TTN','FAT1','MUC16','CLCA4','EGFR','Missense_Mutation','Nonsense_Mutation','Nonstop_Mutation','Splice_Site','age']].values  # Convert to NumPy array

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Define the custom loss function for survival analysis
def cox_loss(y_pred, y):
    hazard_ratio = np.exp(y_pred)
    log_risk = np.log(np.cumsum(hazard_ratio))
    log_lik = y_pred - log_risk
    neg_log_lik = -np.sum(log_lik * y - log_risk * np.sum(y))
    return neg_log_lik

# Initialize the XGBoost regressor
model = xgb.XGBRegressor(objective=cox_loss)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate the concordance index (C-index)
cindex = concordance_index(y_test, -y_pred, np.squeeze(y_test))
print("Concordance Index (C-index):", cindex)


TypeError: cannot unpack non-iterable numpy.float32 object

In [14]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)


X_train shape: (360, 14)
y_train shape: (360, 1)
